In [39]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [40]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [41]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [42]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [43]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [44]:

Base.classes.keys()

['measurement', 'station']

In [45]:
# reflect an existing database into a new model
Measurement = Base.classes.measurement
Station = Base.classes.station

# reflect the tables



In [46]:

session = Session(engine)

In [47]:
first_row_station = session.query(Station).first()
first_row_station.__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x147c8ac0>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397',
 'latitude': 21.2716,
 'elevation': 3.0,
 'id': 1}

In [48]:
# View all of the classes that automap found
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1495b370>,
 'date': '2010-01-01',
 'id': 1,
 'prcp': 0.08,
 'tobs': 65.0,
 'station': 'USC00519397'}

In [49]:
# Save references to each table



In [50]:
# Create our session (link) from Python to the DB


# Exploratory Precipitation Analysis

In [51]:
# Find the most recent date in the data set.

In [64]:
results = session.query(Measurement.date, Measurement.prcp).\
	order_by(Measurement.date.desc()).all()

In [65]:
results = pd.DataFrame(results, columns=("date", 'prcp'))
results

,date,prcp
0,2017-08-23,0.00
1,2017-08-23,0.00
2,2017-08-23,0.08
3,2017-08-23,0.45
4,2017-08-22,0.00
...,...,...
19545,2010-01-01,0.21
19546,2010-01-01,0.15
19547,2010-01-01,0.15
19548,2010-01-01,0.05


In [67]:
first_year = []
for dates in results:
	first_year.append
	if results.loc[results['date'] == '2016-22-08']:
		break

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [11]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number stations in the dataset


In [13]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [16]:
# Close Session
session.close()

NameError: name 'session' is not defined